In [ ]:
import numpy as n
import torch

import sys
sys.path.append('../')

from datasets import load_data
from utils import compute_per_class_AP

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from models import JointDetector
from train_joint import evaluate


# For Joint Model
_, val_loader, obj_class2ind, obj_n_classes, part_class2ind, part_n_classes = load_data('../data/VOCdevkit/VOC2010/', 1, 'animals_train', 'animals_val',
    'animals_object_class2ind', True, True, True, 'animals_part_mergedclass2ind', 0, None)

obj_ind2class = {v: k for k,v in obj_class2ind.items()}
part_ind2class = {v: k for k,v in part_class2ind.items()}

model = JointDetector(obj_n_classes, part_n_classes, 0.9, False).to(device)
model.load_state_dict(torch.load('../saved_model_joint_ft-0.90_animals_train.pth', map_location=device)['model'])

obj_coco_evaluator, part_coco_evaluator, _, _ = evaluate(model, val_loader, device, 500)

print('\nObject classwise AP@IoU=0.5:')
obj_per_class_AP = compute_per_class_AP(obj_coco_evaluator)
for c in range(1, obj_n_classes):
    print('%-5s: %.2f' % (obj_ind2class[c], 100*obj_per_class_AP[c-1]))

print('\nPart classwise AP@IoU=0.5:')
part_per_class_AP = compute_per_class_AP(part_coco_evaluator)
for c in range(1, obj_n_classes):
    print('%-5s: %.2f' % (part_ind2class[c], 100*part_per_class_AP[c-1]))

In [ ]:
from models import get_FasterRCNN_model
from references.detection.engine import evaluate


##### For FasterRCNN Object Detection Model #####
_, val_loader, class2ind, n_classes = load_data('../data/VOCdevkit/VOC2010/', 1, 'animals_train', 'animals_val',
    'animals_object_class2ind', True, False, False, None, 0, None)
model_save_path = '../saved_model_object_animals_train_mAP.pth'

##### For FasterRCNN Part Detection Model #####
# _, val_loader, class2ind, n_classes = load_data('../data/VOCdevkit/VOC2010/', 1, 'animals_train', 'animals_val',
#     'animals_part_mergedclass2ind', False, True, False, None, 0, None)
# model_save_path = '../saved_model_part_animals_train.pth'


ind2class = {v: k for k,v in class2ind.items()}

model = get_FasterRCNN_model(n_classes).to(device)
model.load_state_dict(torch.load(model_save_path, map_location=device)['model'])

coco_evaluator, _ = evaluate(model, val_loader, device, 500)

print('\nClasswise AP@IoU=0.5:')
per_class_AP = compute_per_class_AP(coco_evaluator)
for c in range(1, n_classes):
    print('%-5s: %.2f' % (ind2class[c], 100*per_class_AP[c-1]))